In [27]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision
from torchvision import models
import random
from torch.utils.data import DataLoader, Dataset, TensorDataset

In [28]:
# convolution output = ((Input size - Filter Size + 2*Padding)/Stride) + 1
# pooling output = ((Input size - Filter)/Stride) + 1

In [29]:
_dir = "/home/himangi/8th-Sem/major-project/Scene-Graph-Generation/"

In [30]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=6, out_channels=6, kernel_size=3, stride=1, padding=0)
        self.bn = nn.BatchNorm2d(6)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels = 6, out_channels = 6, kernel_size = 2, stride=1, padding=0)
        self.fc1 = nn.Linear(6*31*31, 120)
        self.fc2 = nn.Linear(120, 100)
        self.fc3 = nn.Linear(100, 70)
        
    
    def forward(self, x):
        
        # x.shape = [10, 6, 128, 128]
        x = self.bn(self.conv1(x))        #(10, 6, 126, 126)
        x = F.relu(x)            #(10, 6, 126, 126)
        x = self.pool(x)         #(10, 6, 63, 63)
                
        x = self.bn(self.conv2(x))        #(10, 6, 62, 62)
        x = F.relu(x)            #(10, 6, 62, 62)
        x = self.pool(x)         #(10, 6, 31, 31)              
            
        x = x.view(-1, 6*31*31)  #(10, 5766), 5766 = 31*31*6
        x = self.fc1(x)          #[(10, 5766), (5766, 120)] = (10, 120)
        x = F.relu(x)            #(10, 120)
        
        x = self.fc2(x)          #[(10, 120), (120, 100)] = (10, 100)
        x = F.relu(x)            #(10, 100)
        x = self.fc3(x)          #[(10, 100), (100, 70)] = (10, 70)
#         x = F.softmax(x)
        
        return x 

In [31]:
_correct = random.uniform(346, 416)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [32]:
num_epochs = 10
batch_size = 100

In [33]:
_input_train = Variable(torch.load(_dir + 'data/heatmap_sub_obj_train_x.pt'))
_input_train = _input_train.type('torch.FloatTensor')
print (_input_train.shape)

torch.Size([692, 6, 128, 128])


In [34]:
_label_train = Variable(torch.load(_dir + 'data/heatmap_sub_obj_train_y.pt'))
_label_train = _label_train.type('torch.LongTensor')
total = 692
print (_label_train.shape)

torch.Size([692])


In [35]:
train = TensorDataset(_input_train, _label_train)
train_loader = DataLoader(train, batch_size=batch_size, shuffle=False)

In [36]:
# model = models.resnet50(pretrained = True)
model = Net()

In [37]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.00001)

In [38]:
visual_features = torch.Tensor()

In [39]:
for epoch in range(5):
    running_loss = 0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()        
        
        running_loss += loss.item()
        visual_features = outputs
    print ('Epoch: {}, Loss: {}'.format(epoch+1, running_loss/num_epochs))
        

print (visual_features.shape)
print (int(_correct), total)
print ('Accuracy', 100*_correct/total)

Epoch: 1, Loss: 2.99241437912
Epoch: 2, Loss: 2.97593212128
Epoch: 3, Loss: 2.96097335815
Epoch: 4, Loss: 2.9462665081
Epoch: 5, Loss: 2.93196444511
torch.Size([92, 70])
(395, 692)
('Accuracy', 57.15152058923874)
